# Specific patient trajectory and linear model
Scripts and methods to plot trajectories for specific patients and compare it to a linear regression model.

This script does:

1. Plots trajectories of specific patients at different diagnosis (CN, MCI, AD). 
2. plot trajectories of patients that convert.
3. Do prediction with a linear model on a small unseen set of patients at different diagnosis, for future times. Show predictions for future, known times.
4. Do the same prediction with a good trained model.

In [1]:
#Import
# working dir
%cd /homedtic/gmarti/CODE/RNN-VAE/

# Imports
import sys
sys.path.insert(0, '/homedtic/gmarti/CODE/RNN-VAE/')
from rnnvae.utils import open_MRI_data_var
from rnnvae import rnnvae
from rnnvae.plot import plot_losses, plot_trajectory, plot_total_loss, plot_z_2d, plot_z_time_2d
import os
import math
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import torch

%matplotlib inline

/homedtic/gmarti/CODE/RNN-VAE


In [2]:
import torch
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
# DEVICE
## Decidint on device on device.
DEVICE_ID = 0
DEVICE = torch.device('cuda:' + str(DEVICE_ID) if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.cuda.set_device(DEVICE_ID)

# out_dir = "experiments/MRI_padding_lin_mask/"
out_dir = "experiments/meta_cort/_h_40_z_7_hid_40_l_1/"

#load parameters
p = eval(open(out_dir + "params.txt").read())

#Seed
torch.manual_seed(p["seed"])
np.random.seed(p["seed"])

p['x_size'] = 68

model = rnnvae.ModelRNNVAE(p["x_size"], p["h_size"], p["hidden"], p["n_layers"], 
                        p["hidden"], p["n_layers"], p["hidden"],
                        p["n_layers"], p["z_dim"], p["hidden"], p["n_layers"],
                        p["clip"], p["n_epochs"], p["batch_size"], DEVICE)
model.load(out_dir+'model.pt')
model = model.to(DEVICE)

For a specific value, generate it for a given number of timepoints and plot it

In [5]:
csv_path = "data/tadpole_cortonly.csv"
X_train, X_test, Y_train, Y_test, mri_col = open_MRI_data_var(csv_path, train_set=0.9, normalize=True, return_covariates=True)

nfeatures = X_train[0].shape[1]

# Apply padding to both X_train and X_val
X_train_tensor = [ torch.FloatTensor(t) for t in X_train ]
X_train_pad = torch.nn.utils.rnn.pad_sequence(X_train_tensor, batch_first=False, padding_value=np.nan)
X_test_tensor = [ torch.FloatTensor(t) for t in X_test ]
X_test_pad = torch.nn.utils.rnn.pad_sequence(X_test_tensor, batch_first=False, padding_value=np.nan)

# Those datasets are of size [Tmax, Batch_size, nfeatures]
mask_train = ~torch.isnan(X_train_pad)
mask_test = ~torch.isnan(X_test_pad)

#convert those NaN to zeros
X_train_pad[torch.isnan(X_train_pad)] = 0
X_test_pad[torch.isnan(X_test_pad)] = 0

max_timepoints = X_train_pad.shape[0]
# Those datasets are of size [Tmax, Batch_size, nfeatures]
# Predict the reconstructions from X_val and X_train
X_test_fwd = model.predict(X_test_pad.to(DEVICE))
X_train_fwd = model.predict(X_train_pad.to(DEVICE))

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
print(X_train_pad.shape)
print(len(mri_col))
print(mri_col)
print(len(X_test))

Get specific patients that we want to study

In [ ]:
# Define patients
# Select the patients from the csv, and see if they are present on the training set
#Candidats:
#CN: 007_S_0068, 002_S_0295
#MCI: 136_S_0107
#AD: 116_S_0487
#Convert MCI to dementia: 007_S_0344. Converteix a m12, el segon timepoint
PTID_CN = "002_S_0295"
PTID_MCI = "136_S_0107"
PTID_AD = "116_S_0487"
PTID_CONVERT = "007_S_0344"

# Compare only baseline
patient_index = [i for i, x in enumerate(Y_train["PTID"]) if x[0] == PTID_CONVERT][0]
#still list of one patient
patient = [X_train[patient_index]]

#ntp train will always be length of the patient minus 1 or 2
#ntp predict will always be the actual full length of the project
ntp_train = patient[0].shape[0] - 1
ntp_predict = patient[0].shape[0]

patient_to_train = [patient[0][:ntp_train, :]]

# Select the feature. We would like to select a feature like the hippocampus or similar
feat = 36
feat_name = mri_col[feat]
print(feat_name)

Use Linear regression for those specific patients, and predict next values

In [ ]:
#Linear model
X_lin = np.array(range(ntp_train)).reshape(-1, 1) # X is the time points
Y_lin = patient_to_train[0]
print(X_lin.shape)
print(Y_lin.shape)
linreg = LinearRegression()
linreg.fit(X_lin, Y_lin)
Y_lin_pred = linreg.predict(np.array(range(ntp_predict)).reshape(-1, 1))
print(Y_lin_pred.shape)

plt.plot(X_lin, Y_lin[:, feat], '-b', label='X (Original)')
#Plot all the objective points
for i in range(ntp_train, ntp_predict):
    plt.plot(i, patient[0][i,feat], 'xb', label='X (to predict)')
plt.plot(list(range(ntp_predict)), Y_lin_pred[:, feat], '-r', label='X (predicted)')
for i in range(ntp_predict):
    plt.plot(i, Y_lin_pred[i,feat], 'xr')

plt.xlabel("time-point")
plt.ylabel("value")
plt.legend(loc='upper right')

plt.title("Linear regression")

In [ ]:
#RNN VAE
#Get tensor for a single subject
# REMEMBER TO PERMUTE, FIRST DIMENSIONS IS TIME!
patient_tensor = torch.FloatTensor(patient_to_train).permute((1,0,2))
X_fwd = model.sequence_predict(patient_tensor.to(DEVICE), ntp_predict)

#Get prediction
X_hat = X_fwd["xnext"]
print(patient_tensor.shape)
print(X_hat.shape)
X_hat_line = X_hat[:, 0, feat]   #Select only the subject we want
X_samples_line = patient_tensor[:, 0, feat].numpy()   #Select only the subject we want
print(X_hat_line.shape)
print(X_samples_line.shape)
# Plot the two lines
plt.figure()
plt.plot(range(len(X_hat_line)), X_hat_line, '-r', label='X (predicted)')
plt.plot(range(len(X_samples_line)), X_samples_line, '-b', label='X (original)')
for i in range(ntp_train, ntp_predict):
    plt.plot(i, patient[0][i,feat], 'xb', label='X (to predict)')

plt.xlabel("time-point")
plt.ylabel("value")

plt.legend(loc='upper left')
plt.title("Predicted vs real")

#Resultats bastant lamentables...

### Generalized prediction
For all subjects, predict the last value, using both a linear model (for each patient) and the RNN-VAE model (fit it here too). Then, predict the last time-point for each patient,
and compute a loss (rmse loss?) over all the predicted values. This way, we would see how the model works and do a major comparison on that.



In [9]:
#parameters
n_to_predict = 1

In [10]:
##Generalized prediction using linear model
# One linear model per subject
Y_pred = []
Y_true = []

#Compute the models and predictions
for p in X_train:
    ntp_train = p.shape[0]-n_to_predict
    if ntp_train == 0:
        continue
    X_lin = np.array(range(ntp_train)).reshape(-1, 1) # X is the time points
    Y_lin = p[:ntp_train, :]
    linreg = LinearRegression()
    linreg.fit(X_lin, Y_lin)
    Y_lin_pred = linreg.predict(np.array(range(ntp_predict)).reshape(-1, 1))
    # Select original and y_lin_pred last point, and append it 
    Y_true.append(p[-1, :])
    Y_pred.append(Y_lin_pred[-1, :])

#Compute mse
mse_linear = mean_squared_error(Y_true, Y_pred)
print(f'Mean Squared Error linear model: {mse_linear}')

Mean Squared Error linear model: 0.4699439642525517


In [11]:
# Predict for max+1 and select only the positions that I am interested in
#this sequence predict DO NOT work well
Y_true = []
Y_pred = []

for i in range(X_train_pad.size(1)):
    x = torch.FloatTensor(X_train[i][:-n_to_predict,:])
    x = x.unsqueeze(1)
    tp = x.size(0) # max time points (and timepoint to predict)
    if tp == 0:
        continue
    X_fwd = model.sequence_predict(x.to(DEVICE), tp+1)
    X_hat = X_fwd['xnext']
    Y_pred.append(X_hat[tp, 0, :]) #get predicted point
    Y_true.append(X_train[i][-1,:])
print(Y_true[0].shape)
print(Y_pred[0].shape)
    
#For each patient in X_hat, saveonly the timepoint that we want
#Compute mse
mse_linear = mean_squared_error(Y_true, Y_pred)
print(f'Mean Squared Error linear model: {mse_linear}')

(40,)
(40,)
Mean Squared Error linear model: 0.3683970790060767


In [12]:
#load parameters
p = eval(open(out_dir + "params.txt").read())
p['x_size'] = 40

##Generalized prediction using RNNVAE
#One general model, and then full prediction

# For the train dataset, we can remove the last timepoint
X_train_tensor = [ torch.FloatTensor(t[:-n_to_predict,:]) for t in X_train ]
X_train_pad = torch.nn.utils.rnn.pad_sequence(X_train_tensor, batch_first=False, padding_value=np.nan)

# no need to do the same on the test set
X_test_tensor = [ torch.FloatTensor(t) for t in X_test ]
X_test_pad = torch.nn.utils.rnn.pad_sequence(X_test_tensor, batch_first=False, padding_value=np.nan)

# Those datasets are of size [Tmax, Batch_size, nfeatures]
# Save mask to unpad later when testing
mask_train = ~torch.isnan(X_train_pad)
mask_test = ~torch.isnan(X_test_pad)

mask_train_tensor = torch.BoolTensor(mask_train)
mask_test_tensor = torch.BoolTensor(mask_test)

#convert those NaN to zeros
X_train_pad[torch.isnan(X_train_pad)] = 0
X_test_pad[torch.isnan(X_test_pad)] = 0

X_train_pad[torch.isnan(X_train_pad)] = 0

max_timepoints = X_train_pad.shape[0]
print(X_train_pad.shape)

# Define model and optimizer

model = rnnvae.ModelRNNVAE(p["x_size"], p["h_size"], p["hidden"], p["n_layers"], 
                        p["hidden"], p["n_layers"], p["hidden"],
                        p["n_layers"], p["z_dim"], p["hidden"], p["n_layers"],
                        p["clip"], p["n_epochs"], p["batch_size"], DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=p["learning_rate"])
model.optimizer = optimizer
model = model.to(DEVICE)

# Fit the model
model.fit(X_train_pad.to(DEVICE), X_test_pad.to(DEVICE), mask_train_tensor.to(DEVICE),  mask_test_tensor.to(DEVICE))

torch.Size([9, 792, 40])
Train loss Epoch:    0/1000 (0%)	Loss: 519.3767	LL: -516.6365	KL: 2.7402	LL/KL: -188.5421
Validation loss Epoch:    0/1000 (0%)	Loss: 443.9859	LL: -442.0950	KL: 1.8909	LL/KL: -233.8074
Train loss Epoch:  100/1000 (10%)	Loss: 246.7952	LL: -229.4583	KL: 17.3370	LL/KL: -13.2352
Validation loss Epoch:  100/1000 (10%)	Loss: 328.6924	LL: -314.9983	KL: 13.6942	LL/KL: -23.0024
Train loss Epoch:  200/1000 (20%)	Loss: 122.4648	LL: -100.7164	KL: 21.7484	LL/KL: -4.6310
Validation loss Epoch:  200/1000 (20%)	Loss: 296.1040	LL: -277.0412	KL: 19.0628	LL/KL: -14.5331
Train loss Epoch:  300/1000 (30%)	Loss: 77.3226	LL: -55.9120	KL: 21.4106	LL/KL: -2.6114
Validation loss Epoch:  300/1000 (30%)	Loss: 281.3371	LL: -260.9074	KL: 20.4297	LL/KL: -12.7710
Train loss Epoch:  400/1000 (40%)	Loss: 39.2923	LL: -16.2076	KL: 23.0847	LL/KL: -0.7021
Validation loss Epoch:  400/1000 (40%)	Loss: 282.7305	LL: -260.5299	KL: 22.2006	LL/KL: -11.7353
Train loss Epoch:  500/1000 (50%)	Loss: 20.5247	L

KeyboardInterrupt: 

In [ ]:
X_train[0][:-1,:].shape